In [1]:
using SparseArrays
using BenchmarkTools
using Statistics
using LinearAlgebra



In [2]:
n_features = 15000
n_examples = 2000
n_classes  = 2

2

In [3]:
function affine_dense_input_sparse(W_dense, b_dense, x_sp)
    n_rows_W, n_cols_W = size(W_dense)
    result = zeros(eltype(W_dense), n_rows_W)
    
    @inbounds for j in 1:n_rows_W
        for i in x_sp.nzind
            result[j] += W_dense[j,i] * x_sp[i] 
        end
        result[j] +=  b[j]
    end
    return result
end

affine_dense_input_sparse (generic function with 1 method)

In [4]:
percentage_sparse = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001]

percent_sparse = percentage_sparse[1]
W = rand(n_features, n_classes);
x_sp = sprand(n_features, percent_sparse);
b = zeros(n_classes);

In [ ]:

@time W'*x_sp

import Base 
import Base.*

#*(W::Adjoint, x::SparseVector) =  mat_dense_vec_sparse(W_dense, x_sp)

In [ ]:
function mat_dense_vec_sparse(W_dense, x_sp)
    n_rows_W, n_cols_W = size(W_dense)
    result = zeros(eltype(W_dense), n_rows_W)
    
    @inbounds for j in 1:n_rows_W
        for i in x_sp.nzind
            result[j] += W_dense[j,i] * x_sp[i] 
        end
    end
    return result
end

In [ ]:
my_prod(W::Adjoint, x::SparseVector) = mat_dense_vec_sparse(W, x) 


In [ ]:
W = rand(n_features, n_classes);
x_sp = sprand(n_features, 0.1);
b = zeros(n_classes);

In [ ]:
typeof(W') <: Adjoint{AbstractFloat, AbstractArray{AbstractFloat,2}}

In [ ]:
typeof(W') <: Adjoint

In [ ]:
typeof(x_sp) <:SparseVector

In [ ]:
@btime my_prod(W',x_sp)

In [ ]:
@btime W'*x_sp

In [ ]:
W'*x_sp + b

In [ ]:
affine_dense_input_sparse(W', b, x_sp)

In [ ]:
time1 = @benchmark W'*x_sp + b;
time2 = @benchmark affine_dense_input_sparse(W', b, x_sp);

result1 = W'*x_sp + b
result2 = affine_dense_input_sparse(W', b, x_sp)

time1_meantime = mean(time1.times)
time2_meantime = mean(time2.times)


print("W'*x_sp + b: ", time1_meantime)
print("\t affine_dense_input_sparse:", time2_meantime)
print("\t isapprox: ", isapprox(result1, result2))


In [ ]:
length(nonzeros(x_sp))/length(x_sp)

In [ ]:
length(x_sp), length(nonzeros(x_sp))

In [ ]:
100*(7512/15000)

In [ ]:
percentage_sparse_tests = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001,0.0001]
results = []

for percent_sparse in percentage_sparse_tests
    W = rand(n_features, n_classes);
    x_sp = sprand(n_features, percent_sparse);
    b = zeros(n_classes);
    
    time1 = @benchmark W'*x_sp + b;
    time2 = @benchmark affine_dense_input_sparse(W', b, x_sp);

    result1 = W'*x_sp + b
    result2 = affine_dense_input_sparse(W', b, x_sp)

    time1_meantime = Int(round(mean(time1.times)))
    time2_meantime = Int(round(mean(time2.times)))
    improvement    = round(time1_meantime/ time2_meantime, digits = 2)
    percentage_nonzeros = round(100*(length(nonzeros(x_sp))/n_features), digits=2)
    
    x = (percentage_sparse=percentage_nonzeros, t1=time1_meantime, t2=time2_meantime, improvement=improvement)
    push!(results, x)
    
    println("\nTrue % nonzeros:", percentage_nonzeros, "\t percent_sparse given:", 100*percent_sparse)

    print("Improvement: ", improvement,"x    ")
    print("\tW'*x_sp + b: ", time1_meantime)
    print("\tCustom:", time2_meantime)
    print("\tisapprox: ", isapprox(result1, result2))
    println(" ")

end

It is not worth it Weights are not adjoint

In [ ]:
percentage_sparse_tests = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001,0.0001]
results = []
W_t = copy(W')
for percent_sparse in percentage_sparse_tests
    W = rand(n_features, n_classes);
    x_sp = sprand(n_features, percent_sparse);
    b = zeros(n_classes);
    
    time1 = @benchmark W_t*x_sp + b;
    time2 = @benchmark affine_dense_input_sparse(W_t, b, x_sp);

    result1 = W_t*x_sp + b
    result2 = affine_dense_input_sparse(W_t, b, x_sp)

    time1_meantime = Int(round(mean(time1.times)))
    time2_meantime = Int(round(mean(time2.times)))
    improvement    = round(time1_meantime/ time2_meantime, digits = 2)
    percentage_nonzeros = round(100*(length(nonzeros(x_sp))/n_features), digits=2)
    
    x = (percentage_sparse=percentage_nonzeros, t1=time1_meantime, t2=time2_meantime, improvement=improvement)
    push!(results, x)
    
    println("\nTrue % nonzeros:", percentage_nonzeros, "\t percent_sparse given:", percent_sparse)

    print("Improvement: ", improvement,"x    ")
    print("\tW_t*x_sp + b: ", time1_meantime)
    print("\tCustom:", time2_meantime)
    print("\tisapprox: ", isapprox(result1, result2))
    println(" ")

end

### Only Matrix vector product

In [5]:
function mat_dense_vec_sparse(W_dense, x_sp)
    n_rows_W, n_cols_W = size(W_dense)
    result = zeros(eltype(W_dense), n_rows_W)
    
    @inbounds for j in 1:n_rows_W
        for i in x_sp.nzind
            result[j] += W_dense[j,i] * x_sp[i] 
        end
    end
    return result
end

mat_dense_vec_sparse (generic function with 1 method)

In [6]:
percentage_sparse_tests = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001,0.0001]
results = []

for percent_sparse in percentage_sparse_tests
    W    = rand(n_features, n_classes);
    x_sp = sprand(n_features, percent_sparse);
    b    = zeros(n_classes);
    
    time1 = @benchmark W'*x_sp;
    time2 = @benchmark mat_dense_vec_sparse(W', x_sp);

    result1 = W'*x_sp + b
    result2 = mat_dense_vec_sparse(W', x_sp)

    time1_meantime = Int(round(mean(time1.times)))
    time2_meantime = Int(round(mean(time2.times)))
    improvement    = round(time1_meantime/ time2_meantime, digits = 2)
    percentage_nonzeros = round(100*(length(nonzeros(x_sp))/n_features), digits=2)
    
    x = (percentage_sparse=percentage_nonzeros, t1=time1_meantime, t2=time2_meantime, improvement=improvement)
    push!(results, x)
    
    println("\nTrue % nonzeros:", percentage_nonzeros, "\t percent_sparse given:", percent_sparse)

    print("Improvement: ", improvement,"x    ")
    print("\tW'*x_sp : ", time1_meantime)
    print("\tCustom:", time2_meantime)
    print("\tisapprox: ", isapprox(result1, result2))
    println(" ")

end


True % nonzeros:50.21	 percent_sparse given:0.5
Improvement: 1.92x    	W'*x_sp : 1123960	Custom:584711	isapprox: true 

True % nonzeros:39.62	 percent_sparse given:0.4
Improvement: 2.2x    	W'*x_sp : 1023303	Custom:465889	isapprox: true 

True % nonzeros:29.9	 percent_sparse given:0.3
Improvement: 2.64x    	W'*x_sp : 942773	Custom:356707	isapprox: true 

True % nonzeros:19.88	 percent_sparse given:0.2
Improvement: 3.21x    	W'*x_sp : 740356	Custom:230381	isapprox: true 

True % nonzeros:10.25	 percent_sparse given:0.1
Improvement: 4.62x    	W'*x_sp : 565736	Custom:122556	isapprox: true 

True % nonzeros:1.0	 percent_sparse given:0.01
Improvement: 103.09x    	W'*x_sp : 291132	Custom:2824	isapprox: true 

True % nonzeros:0.06	 percent_sparse given:0.001
Improvement: 522.89x    	W'*x_sp : 116081	Custom:222	isapprox: true 

True % nonzeros:0.0	 percent_sparse given:0.0001
Improvement: 452.99x    	W'*x_sp : 43034	Custom:95	isapprox: true 


In [32]:
typeof(W')

Adjoint{Float64,Array{Float64,2}}

In [41]:
W    = rand(15000, 10);
x_sp = sprand(15000,0.1);

In [42]:
@btime W'*x_sp;

  2.663 ms (10 allocations: 752 bytes)


In [22]:
typeof(x_sp)

SparseVector{Float64,Int64}

In [24]:
x_sp

15000-element SparseVector{Float64,Int64} with 1464 stored entries:
  [16   ]  =  0.498027
  [44   ]  =  0.975501
  [49   ]  =  0.307525
  [54   ]  =  0.0203444
  [61   ]  =  0.146972
  [63   ]  =  0.69342
  [64   ]  =  0.630906
  [68   ]  =  0.153501
  [74   ]  =  0.383094
  [83   ]  =  0.434311
           ⋮
  [14904]  =  0.146065
  [14918]  =  0.446119
  [14924]  =  0.128669
  [14928]  =  0.957264
  [14933]  =  0.937201
  [14949]  =  0.391561
  [14967]  =  0.0975735
  [14972]  =  0.278263
  [14979]  =  0.231342
  [14996]  =  0.548058
  [14997]  =  0.319715

In [23]:
@btime W'*x_sp

  533.952 μs (6 allocations: 304 bytes)


2-element SparseVector{Float64,Int64} with 2 stored entries:
  [1]  =  357.016
  [2]  =  361.428

In [25]:
Wt= W'

2×15000 Adjoint{Float64,Array{Float64,2}}:
 0.93468   0.825887  0.919829  0.032798  …  0.724829  0.829388  0.239847
 0.859358  0.922824  0.699805  0.44007      0.889028  0.110043  0.275656

In [26]:
@btime Wt*x_sp

  534.485 μs (5 allocations: 288 bytes)


2-element SparseVector{Float64,Int64} with 2 stored entries:
  [1]  =  357.016
  [2]  =  361.428

In [31]:
@btime mat_dense_vec_sparse(W',x_sp)

  110.527 μs (2 allocations: 112 bytes)


2-element Array{Float64,1}:
 357.0155355144745 
 361.42813947591645

In [ ]:
percentage_sparse_tests = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001,0.0001]
results = []
W_t = copy(W')


for percent_sparse in percentage_sparse_tests
    W = rand(n_features, n_classes);
    x_sp = sprand(n_features, percent_sparse);
    b = zeros(n_classes);
    
    time1 = @benchmark W_t*x_sp;
    time2 = @benchmark mat_dense_vec_sparse(W_t, x_sp);

    result1 = W_t*x_sp + b
    result2 = mat_dense_vec_sparse(W_t, x_sp)

    time1_meantime = Int(round(mean(time1.times)))
    time2_meantime = Int(round(mean(time2.times)))
    improvement    = round(time1_meantime/ time2_meantime, digits = 2)
    percentage_nonzeros = round(100*(length(nonzeros(x_sp))/n_features), digits=2)
    
    x = (percentage_sparse=percentage_nonzeros, t1=time1_meantime, t2=time2_meantime, improvement=improvement)
    push!(results, x)
    
    println("\nTrue % nonzeros:", percentage_nonzeros, "\t percent_sparse given:", percent_sparse)

    print("Improvement: ", improvement,"x    ")
    print("\tW'*x_sp : ", time1_meantime)
    print("\tCustom:", time2_meantime)
    print("\tisapprox: ", isapprox(result1, result2))
    println(" ")

end

### Overloading the `*` operator

In [ ]:
using LinearAlgebra

In [ ]:
function mat_dense_vec_sparse(W_dense, x_sp)
    n_rows_W, n_cols_W = size(W_dense)
    result = zeros(eltype(W_dense), n_rows_W)
    
    @inbounds for j in 1:n_rows_W
        for i in x_sp.nzind
            result[j] += W_dense[j,i] * x_sp[i] 
        end
    end
    return result
end

In [ ]:
@time W'*x_sp

In [ ]:
import Base 
import Base.*

#*(W::Adjoint, x::SparseVector) =  mat_dense_vec_sparse(W_dense, x_sp)

In [ ]:
*(W::Adjoint, x::SparseVector) = mat_dense_vec_sparse(W, x) 

In [ ]:
@time mat_dense_vec_sparse(W',x_sp)

In [ ]:
@time W'*x_sp

In [ ]:
myprod2(W::Adjoint, x::SparseVector) = mat_dense_vec_sparse(W, x) 

In [ ]:
@time myprod2(W',x_sp)

In [ ]:
*(W::Adjoint, x::SparseVector) = mat_dense_vec_sparse(W, x) 

In [ ]:
@time W'*x_sp

In [ ]:
typeof(W')

In [ ]:
my_prod(W',x_sp)

In [ ]:
*(W::Adjoint{T, AbstractArray{T,2}}, x::SparseVector{T,I}) where {T<:AbstractFloat, I<:Int} = mat_dense_vec_sparse(W_dense, x_sp) 

In [ ]:
typeof(W'), typeof(x_sp)

In [ ]:
W'*x_sp

In [ ]:
@time W'*x_sp

In [ ]:
@which W'*x_sp

In [ ]:
@time  W'*x_sp

In [ ]:
@time mat_dense_vec_sparse(W', x_sp)

**A model is an object storing hyperparameters associated with some machine learning algorithm. In MLJ, hyperparameters include configuration parameters, like the number of threads, and special instructions, such as "compute feature rankings", which may or may not affect the final learning outcome. However, the logging level (verbosity below) is excluded.
**

In my case I coded the model as a struct containing hyperparameters as well as the number of classes and the number of features
```
mutable struct MulticlassPerceptronClassifier{T}
    W::AbstractMatrix{T}
    b::AbstractVector{T}
    n_classes::Int
    n_features::Int
end
```

In MLJ I would do

```
mutable struct MulticlassPerceptronClassifier <: MLJBase.Deterministic
    weight_average::Bool
end
```

**
Models (which are mutable) should not be given internal constructors. It is recommended that they be given an external lazy keyword constructor of the same name. This constructor defines default values for every field, and optionally corrects invalid field values by calling a clean! method (whose fallback returns an empty message string):
**




Native implementations (preferred option). The implementation code lives in the same package that contains the learning algorithms implementing the interface.



In this case, it is sufficient to open an issue at MLJRegistry requesting the package to be registered with MLJ. Registering a package allows the MLJ user to access its models' metadata and to selectively load them.



Additionally, one needs to ensure that the implementation code defines the package_name and load_path model traits appropriately, so that MLJ's @load macro can find the necessary code (see MLJModels/src for examples). The @load command can only be tested after registration. If changes are made, lodge an issue at MLJRegistry to make the changes available to MLJ.



### Adding a model to MLJ


#### `fit!(machine, rows, verbosity ) interface `


#### `fit(model, verbosity, X[rows], y[rows])` 


#### `fit(model, verbosity, X[rows], y[rows])` 

